# Steps to Use the Model in a Real-World Application

## 1. Convert the Model into a Predictive API

### Deploying the model as an API will allow real-time or batch predictions.

#### Options:
- **FastAPI** (recommended for speed & scalability)
- **Flask** (simpler but slower)
- **Django REST Framework** (if using Django-based apps)

## 2. Save and Load the Model

In [ ]:

import joblib
import tensorflow as tf

# Save XGBoost model
joblib.dump(xgb_model, "xgb_model.pkl")

# Save LSTM model
lstm_model.save("lstm_model.h5")

# Load models later:
xgb_model = joblib.load("xgb_model.pkl")
lstm_model = tf.keras.models.load_model("lstm_model.h5")


## 3. Create a FastAPI Backend

In [ ]:

from fastapi import FastAPI
import joblib
import tensorflow as tf
import numpy as np
from pydantic import BaseModel

# Load models
xgb_model = joblib.load("xgb_model.pkl")
lstm_model = tf.keras.models.load_model("lstm_model.h5")

app = FastAPI()

# Define request schema
class PowerInput(BaseModel):
    features: list  # List of numerical feature values

@app.post("/predict")
def predict_power(input_data: PowerInput):
    X_input = np.array(input_data.features).reshape(1, -1)
    
    # XGBoost prediction
    xgb_pred = xgb_model.predict(X_input)
    
    # Prepare input for LSTM (reshape to 3D)
    X_input_lstm = np.reshape(X_input, (1, 1, X_input.shape[1]))
    lstm_pred = lstm_model.predict(X_input_lstm).flatten()
    
    # Final power consumption prediction
    final_pred = xgb_pred + lstm_pred
    return {"predicted_power_consumption": final_pred.tolist()}


### Run API:
```bash
uvicorn app:app --host 0.0.0.0 --port 8000
```

## 4. Deploy to the Cloud

#### Deployment Options:
- **AWS Lambda + API Gateway** (serverless)
- **Google Cloud Run** (auto-scaling)
- **Docker + Kubernetes** (production-ready)

### Example Dockerfile:

In [ ]:

FROM python:3.9
WORKDIR /app
COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt
COPY . .
CMD ["uvicorn", "app:app", "--host", "0.0.0.0", "--port", "8000"]


### Build and Run Docker:
```bash
docker build -t power-prediction .
docker run -p 8000:8000 power-prediction
```

## 5. Create a Dashboard (Optional)

In [ ]:

import streamlit as st
import requests

st.title("Power Consumption Prediction")

features = st.text_input("Enter feature values (comma-separated)")
if st.button("Predict"):
    data = {"features": list(map(float, features.split(",")))}
    response = requests.post("http://localhost:8000/predict", json=data)
    st.write("Predicted Power Consumption:", response.json()["predicted_power_consumption"])


### Run Streamlit:
```bash
streamlit run dashboard.py
```